In [1]:
import numpy as np
import cv2


In [2]:
# load data 
without_mask=np.load('With_out_mask1.npy')
with_mask=np.load('With_mask1.npy')

In [3]:
without_mask.shape

(200, 50, 50, 3)

In [4]:
with_mask.shape

(200, 50, 50, 3)

Now we can see that data is loaded with the shape 200, 50, 50, 3.

Here 200 is the number of images we have collected
50, 50 is the size of each image
3 is the color channel (red, green, blue)
We can reshape the data to make it 2D :

In [5]:
# convert four dimention data into two dimentional
with_mask=with_mask.reshape(200,50*50*3)
without_mask=without_mask.reshape(200,50*50*3)

In [6]:
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

we will concatenate the data into a single array

In [9]:
# concate the two datasets 
X=np.r_[with_mask,without_mask]

In [10]:
X.shape

(400, 7500)

In [11]:
# label datasets by using 0 and 1.0
labels=np.zeros(X.shape[0]) # we are converting all 400 labels as 0

In [12]:
labels[200:]=1.0 # then last 200 convet in 1.0 

In [59]:
# we apply as name
names={0:'Mask',1:'No Mask'}

In [ ]:
# We use svm algorithum to classification task

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [16]:
from sklearn.model_selection import train_test_split

In [47]:
x_train,x_test,y_train,y_test=train_test_split(X,labels,test_size=0.30)

In [48]:
x_train.shape

(280, 7500)

In [49]:
# we have lot of columns they slow the classification machine larning algorithum that why we use PCA method to reduce dimention of datasets

In [50]:
#Dimensionality Reuction
from sklearn.decomposition import PCA 
pca=PCA(n_components=3)
x_train=pca.fit_transform(x_train)

In [51]:
x_train.shape

(280, 3)

In [52]:
x_train[0]

array([-3682.37605071,  3853.51362465,  1936.58635487])

In [53]:
# Eigen value / Eigen vectore

In [54]:
svm=SVC()
svm.fit(x_train,y_train)

SVC()

In [55]:
# we also reduce the dimention of test datasets 
x_test=pca.transform(x_test)

we will concatenate the data into a single array

In [56]:
y_pred=svm.predict(x_test)

In [57]:
# Now we chake the accuracy of over model
accuracy_score(y_test,y_pred)

0.9916666666666667

In [45]:
# perfomace 
confusion_matrix(y_test,y_pred)

array([[52,  1],
       [ 0, 60]], dtype=int64)

In [58]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99        57
         1.0       0.98      1.00      0.99        63

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



we can test our faces with or without mask and check whether this algorithm is able to identity you that you are wearing a mask or not.

In [ ]:
face_cascade = cv2.CascadeClassifier("C:\\Users\\Ankush Niwane\\Downloads\\machine_larning program\\haar-cascade-files-master\\haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(0)
font=cv2.FONT_HERSEY_COMPLEX
while True:
    ret, img = cap.read()
    if ret:
        
        faces = face_cascade.detectMultiScale(img)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            face=img[y:y+h,x:x+w,:]
            face=cv2.resize(face,(50,50))
            face=face.reshape(1,-1)
            face=pca.transform(face)
            pred=svm.predict(face)[0]
            n=names[int(pred)]
            cv2.putText(img,n,(x,y),font,1,(255,0,0),2)
            print(n)
            
            
        cv2.imshow('img',img)
        if cv2.waitKey(2)==27 :
            break
        

cap.release()
cv2.destroyAllWindows()